In [ ]:
!pip install fastai --upgrade -q
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from IPython.display import display,HTML

from fastai.text.all import *
path = untar_data(URLs.IMDB)

# Natural Language Understanding
Verarbeitung von Texten mithilfe von fastai und deep learning Modellen.

## Tokenization mithilfe von fastai
* word-based
* subword-based
* character-based

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
txt = files[0].open().read(); txt[:75]

'Thankfully I watched this film alone, enabling me to fast-forward through t'

### Word based Tokenization

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#170) ['Thankfully','I','watched','this','film','alone',',','enabling','me','to','fast','-','forward','through','the','worst','scenes','(','aka','most','of','the','film',',','actually',')','.','OK',',','some'...]


In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))
# BOS = beginning of stream

(#182) ['xxbos','xxmaj','thankfully','i','watched','this','film','alone',',','enabling','me','to','fast','-','forward','through','the','worst','scenes','(','aka','most','of','the','film',',','actually',')','.','xxup','ok'...]


spezielle tokens:
* xxbos: Beginn des Textes
* xxmaj: Das nächste Wort beginnt mit einem Großbuchstaben
* xxunk: unbekanntes Wort.

In [ ]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### SubWord based Tokenization

In [ ]:
txts = L(o.open().read() for o in files[:2000])

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [ ]:
subword(1000)

'▁Th an k ful ly ▁I ▁watch ed ▁this ▁film ▁a lo ne , ▁en ab l ing ▁me ▁to ▁f as t - for ward ▁through ▁the ▁worst ▁scenes ▁( ak a ▁most ▁of ▁the ▁film , ▁actually ).'

In [ ]:
subword(200)

'▁ T h an k f ul ly ▁I ▁w at ch ed ▁this ▁film ▁ al on e , ▁ en a b l ing ▁ m e ▁to ▁f as t - f or w ar d ▁'

In [ ]:
subword(10000)

'▁Thankfully ▁I ▁watched ▁this ▁film ▁alone , ▁enabl ing ▁me ▁to ▁fast - forward ▁through ▁the ▁worst ▁scenes ▁( aka ▁most ▁of ▁the ▁film , ▁actually ). ▁OK , ▁some ▁of ▁it ▁is ▁not ▁all ▁bad , ▁with ▁partial ly'

## Numericalization

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#182) ['xxbos','xxmaj','thankfully','i','watched','this','film','alone',',','enabling','me','to','fast','-','forward','through','the','worst','scenes','(','aka','most','of','the','film',',','actually',')','.','xxup','ok'...]


In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#182) ['xxbos','xxmaj','thankfully','i','watched','this','film','alone',',','enabling'...]

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)
# defaults: min_freq=3,max_vocab=60000

"(#1888) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','it','i','is','in'...]"

In [ ]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,    0,   17,  442,   20,   30, 1128,   11,    0,   78,   15,  796,   25,  706,  181,    9,  164,  157,   37])

In [ ]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj xxunk i watched this film alone , xxunk me to fast - forward through the worst scenes ('

## Arbeiten mit Batches und Text

In [ ]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
1,movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
2,first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
3,how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
4,of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
5,will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df1 = pd.DataFrame(d_tokens)

d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df2 = pd.DataFrame(d_tokens)

d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df3 = pd.DataFrame(d_tokens)

print(df1)
print(df2)
print(df3)


       0        1             2        3        4
0  xxbos    xxmaj            in     this  chapter
1  movie  reviews            we  studied       in
2  first       we          will     look       at
3    how       to     customize       it        .
4     of      the  preprocessor     used       in
5   will    study           how       we    build
         0           1      2          3       4
0        ,          we   will         go    back
1  chapter           1    and        dig  deeper
2      the  processing  steps  necessary      to
3    xxmaj          by  doing       this       ,
4      the        data  block       xxup     api
5        a    language  model        and   train
         0     1        2        3            4
0     over   the  example       of  classifying
1    under   the  surface        .        xxmaj
2  convert  text     into  numbers          and
3       we   'll     have  another      example
4        .    \n    xxmaj     then           we
5       it   for   

**Vorgehensweise:**
1. Konvertierung vom Text in einen Stream. Zufällige Durchmischung der Einträge (nicht der Reihenfolge der Wörter)
1. Aufteilen des Streams in eine bestimmte Anzahl an Batches. Die Reihenfolge der Tokens muss beibehalten werden. Zu Beginn der Verarbeitung wird das spezielle Starttoken (`XXBOS`) hinzugefügt.

In [ ]:
nums200 = toks200.map(num)

In [ ]:
dl = LMDataLoader(nums200)

In [ ]:
x,y = first(dl)
x.shape,y.shape

((64, 72), (64, 72))

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj xxunk i watched this film alone , xxunk me to fast - forward through the worst scenes ('

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj xxunk i watched this film alone , xxunk me to fast - forward through the worst scenes ( xxunk'

## Trainieren des Sprachmodells

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj this movie came from out of nowhere ( as did xxmaj michael xxmaj keaton ) to be one of the funniest movies around . xxmaj keaton 's character of xxmaj billy xxmaj blaze was so over the top funny that it was worth seeing over and over and over again . xxmaj perfectly countered by xxmaj henry xxmaj xxunk low key xxmaj chuck . \n\n xxmaj the supports characters were great as well - the future xxunk ,","xxmaj this movie came from out of nowhere ( as did xxmaj michael xxmaj keaton ) to be one of the funniest movies around . xxmaj keaton 's character of xxmaj billy xxmaj blaze was so over the top funny that it was worth seeing over and over and over again . xxmaj perfectly countered by xxmaj henry xxmaj xxunk low key xxmaj chuck . \n\n xxmaj the supports characters were great as well - the future xxunk , the"
1,"hardly ended with those characters . a phenomenal cast was wasted on a weak script . xxmaj the morals of the story were xxup pc to the max . xxmaj there were a few clever twists but not nearly enough . xxmaj the dialouge was embarrassing at times . xxmaj it was n't all bad.i just ca n't believe this movies high score so far . xxmaj it was somewhat entertaining , just a little insulting to ones intelligence .","ended with those characters . a phenomenal cast was wasted on a weak script . xxmaj the morals of the story were xxup pc to the max . xxmaj there were a few clever twists but not nearly enough . xxmaj the dialouge was embarrassing at times . xxmaj it was n't all bad.i just ca n't believe this movies high score so far . xxmaj it was somewhat entertaining , just a little insulting to ones intelligence . i"


### Finetuning

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.125159,3.911799,0.300254,49.988823,26:33


In [ ]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.880344,3.775839,0.317084,43.634087,28:43
1,3.811964,3.711299,0.324937,40.906929,28:39
2,3.753009,3.658600,0.330815,38.806969,28:50
3,3.669421,3.627583,0.334454,37.621784,28:43
4,3.608968,3.601918,0.337182,36.668488,28:55
5,3.559628,3.584454,0.339622,36.033688,29:10


Buffered data was truncated after reaching the output size limit.

In [ ]:
learn.save_encoder('finetuned')

### Text-Generation mit dem Sprachmodell

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

i liked this movie because it was a good documentary . It is a very powerful movie . There are many people coming out of the United States with the knowledge that it is an amazing movie . This movie
i liked this movie because it did n't quite capture Christopher Lee as a person . He was a great actor and seen the Beatles , and they had been the ones that he wanted to watch . And i


## Die Classifier DataLoaders erstellen

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
nums_samp = toks200[:10].map(num)

In [ ]:
nums_samp.map(len)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))